In [1]:
from submodules import Objects, Robots, Tasks, Users
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

Firstly, let's go through iteration cycle
## Create scene

In [2]:
s = Scene(init='drawer_and_cups')
s.cup1.position = s.r.eef_position

Configuration of gestures and actions

In [3]:
A = Actions.A
G = Gestures.G
U = Users.U
UID = Users.selected_id
print(A)
print(G)
print(U)
print(UID)

['move_up', 'open', 'put', 'pour', 'close', 'pick_up']
['swipe_up', 'open', 'swipe_back', 'hold', 'rotate', 'swipe_forward']
['Jan', 'Mara']
0


## Generate policy

In [4]:
# Static parmeters
TaTo = ('open', 'drawer')
scene_state = s.generate_scene_state(A, G, U, UID, TaTo)

bn = MappingBayesNet(scene_state)

print(bn.create_observation())
bn.init_policy_simple()

{'focus_point': array([[2.15884208, 0.02928376, 0.22882041]]), 'gesture_vec': array([0.0511121 , 0.70777622, 0.06689742, 0.10636917, 0.0456149 ,
       0.02223019])}


{'CM_est': array([[1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]])}

## Create target state

In [5]:
s2 = Scene(init='from_dict', import_data=scene_state)
s2.drawer.open()
s2.drawer.put_in(s2.cup1)
s2.cup1.gripper_move(s2.drawer.position)
s2.object_positions

[array([2, 0, 0]), array([3, 0, 3]), array([2, 2, 0])]

## Select action

In [6]:
action = bn.select_action()
action

{'target_object': 'drawer', 'target_action': 'open'}

## New observation / new world state
- Updates the scene


In [7]:
# temporary place
Actions.do(s, action, ignore_location=True)
s.drawer.info

('open', 'drawer') done!
drawer, drawer, [2 0 0], opened, []


# Reward computation

In [8]:
s == s2

True

# Easy example: Open drawer, put there a cup

In [9]:
action_sequence = Users().demo__put_to_drawer()
for action_input in action_sequence:
    scene_state = s.generate_scene_state(A, G, U, UID, TaTo)
    scene_state['TA'] = action_input[0]
    scene_state['TO'] = action_input[1]
    
    bn = MappingBayesNet(scene_state)
    bn.create_observation()
    bn.init_policy_simple()
    action = bn.select_action()
    print(action)
    
    Actions.do(s, action)
    
    reward = s == s2
    print(f"Reward: {reward}")


{'target_object': 'drawer', 'target_action': 'open'}
Reward: True
{'target_object': 'cup1', 'target_action': 'pick_up'}
('pick_up', 'cup1') done!
Reward: True
{'target_object': 'drawer', 'target_action': 'put'}
Reward: True


# Generating observations

In [10]:
s = Scene(init='drawer_and_cups', random=True)
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], opened, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 3 3], opened, rotation: 1, attached: -


In [11]:
s = Scene(init='drawer_and_cups', random=False)
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


# Apply actions on scene

In [12]:
Actions.do(s, ('open', 'drawer'))  
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [13]:
Actions.do(s, ('close', 'drawer'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [14]:
Actions.do(s, ('pick_up', 'cup1'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [15]:
A

['move_up', 'open', 'put', 'pour', 'close', 'pick_up']

In [16]:
G

['swipe_up', 'open', 'swipe_back', 'hold', 'rotate', 'swipe_forward']

In [17]:
s.O

['drawer', 'cup1', 'cup2']

In [18]:
Actions.do(s, ('put', 'drawer'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [19]:
Actions.do(s, ('open', 'drawer'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [20]:
Actions.do(s, ('put', 'drawer'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [21]:
Actions.do(s, ('pick_up', 'cup2'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [22]:
Actions.do(s, ('put', 'drawer'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [23]:
s = Scene(init='drawer,cup,cup')
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [3 0 0], opened, []
cup, cup, [3 1 0], empty
cup1, cup, [2 2 0], empty
Robot: [2 3 3], opened, rotation: 1, attached: -


In [24]:
Actions.do(s, ('pick_up', 'cup'))

False

In [25]:
Actions.do(s, ('open', 'drawer'))

False

In [26]:
Actions.do(s, ('put', 'drawer'))

False

In [27]:
Actions.do(s, ('close', 'drawer'))

False

In [28]:
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [3 0 0], opened, []
cup, cup, [3 1 0], empty
cup1, cup, [2 2 0], empty
Robot: [2 3 3], opened, rotation: 1, attached: -


# Move tests

In [29]:
s = Scene(init='cup')
print(s)

Scene info. shape: [4, 4, 4]
cup, cup, [3 1 0], full
Robot: [0 1 3], closed, rotation: 0, attached: -


In [30]:
s.r.eef_position = s.cup.position

In [31]:
Actions.do(s, ('pick_up','cup'))
s.info

('pick_up', 'cup') done!
Scene info. shape: [4, 4, 4]
cup, cup, [3 1 0], full
Robot: [3 1 0], closed, rotation: 0, attached: cup


In [32]:
s = Scene(init='cup,cup')
s.r.eef_position = s.cup.position
Actions.do(s, ('pick_up', 'cup'))

('pick_up', 'cup') done!


True

In [33]:
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [3 0 0], full
cup1, cup, [1 2 0], full
Robot: [3 0 0], opened, rotation: 0, attached: cup


In [34]:
Actions.do(s, ('move_up',None))

('move_up', None) done!


True

In [35]:
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [3 0 1], full
cup1, cup, [1 2 0], full
Robot: [3 0 1], opened, rotation: 0, attached: cup


# Generating data

In [36]:
# Generate random scene
scenes = []
scenes.append(Scene(init='drawer,cup,cup'))
s = scenes[0]
scenes[0].info
s.O

Scene info. shape: [4, 4, 4]
drawer, drawer, [3 1 0], closed, []
cup, cup, [1 3 0], empty
cup1, cup, [1 0 0], full
Robot: [0 0 3], closed, rotation: 0, attached: -


['drawer', 'cup', 'cup1']

In [37]:
(Actions.get_random_action(), s.get_random_object())

('put', 'cup')

In [38]:
TaTos = []
# Generate random action
for i in range(10):
    while True:
        TaTo = (Actions.get_random_action(), s.get_random_object())
        if Actions.do(s, TaTo, ignore_location=True):
            TaTos.append(TaTo)
            break

('open', 'drawer') done!
('pick_up', 'cup') done!
('open', 'drawer') done!
('put', 'drawer') done!
('close', 'drawer') done!
('open', 'drawer') done!
('close', 'drawer') done!
('pick_up', 'cup1') done!
('pour', 'drawer') done!
('open', 'drawer') done!


In [39]:
TaTos

[('open', 'drawer'),
 ('pick_up', 'cup'),
 ('open', 'drawer'),
 ('put', 'drawer'),
 ('close', 'drawer'),
 ('open', 'drawer'),
 ('close', 'drawer'),
 ('pick_up', 'cup1'),
 ('pour', 'drawer'),
 ('open', 'drawer')]

In [40]:
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [3 1 0], opened, ['cup']
cup, cup, [1 3 0], empty
cup1, cup, [1 0 0], empty
Robot: [0 0 3], closed, rotation: 0, attached: cup1


In [41]:
s.cup.print_structure()

Structure:
[cup]


In [42]:
s.cup1.print_structure()

Structure:
[cup1]


In [43]:
TaTo = (Actions.get_random_action(), s.get_random_object())
Actions.do(s, TaTo, ignore_location=True)

False

In [44]:
TaTo = ('pick_up', 'cup')
Actions.do(s, TaTo, ignore_location=True)

False

In [45]:
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [3 1 0], opened, ['cup']
cup, cup, [1 3 0], empty
cup1, cup, [1 0 0], empty
Robot: [0 0 3], closed, rotation: 0, attached: cup1


In [46]:
TaTo = ('put_on_target', 'cup1')
Actions.do(s, TaTo, ignore_location=True)

('put_on_target', 'cup1') done!


True

In [47]:
TaTos = []
# Generate random action
for i in range(10):
    j = 0
    while True:
        TaTo = (Actions.get_action(j//len(A)), s.get_object(j%len(s.O)))
        if Actions.do(s, TaTo, ignore_location=True):
            TaTos.append(TaTo)
            break
        j += 1
        if j == 64: raise Exception("out of options")
            

('open', 'drawer') done!
('open', 'drawer') done!
('open', 'drawer') done!
('open', 'drawer') done!
('open', 'drawer') done!
('open', 'drawer') done!
('open', 'drawer') done!
('open', 'drawer') done!
('open', 'drawer') done!
('open', 'drawer') done!


In [48]:
from copy import deepcopy
s.cup1.fill()
s1 = s.copy()

In [49]:
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [3 1 0], opened, ['cup']
cup, cup, [1 3 0], empty
cup1, cup, [1 1 0], full
Robot: [0 0 3], closed, rotation: 0, attached: -


In [50]:
s1.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [3 1 0], opened, ['cup']
cup, cup, [1 3 0], empty
cup1, cup, [1 1 0], full
Robot: [0 0 3], closed, rotation: 0, attached: -


# Attempt 1: Loop with reward

In [53]:
A = Actions.A
G = Gestures.G
U = Users.U
UID = Users.selected_id

# hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

In [54]:
for i in range(1,10):
    epochs, penalties, reward, = 0, 0, 0
    done = False

    s = Scene(init='drawer', random=False)
    s2 = s.copy()
    s2.drawer.open()

    state = s.generate_scene_state(A, G, U, UID, ('open', 'drawer'))
    bn = MappingBayesNet(state)

    while not done:
        if np.random.uniform(0,1) < epsilon:
            action = (Actions.get_random_action(), 'drawer')
        else:
            bn.create_observation()
            action = bn.select_action()
        print(f"Action: {action}")
        s, reward, done = Actions.step(s, s2, action)

        bn.policy_update(reward)

Action: {'target_object': 'drawer', 'target_action': 'open'}
('open', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'open'}
('open', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'open'}
('open', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'open'}
('open', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'open'}
('open', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'open'}
('open', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'open'}
('open', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'open'}
('open', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'open'}
('open', 'drawer') done!


In [55]:
A = Actions.A = ['move_up', 'move_down', 'move_left', 'move_right', 'open', 'close']

for i in range(1,10):
    epochs, penalties, reward, = 0, 0, 0
    done = False

    s = Scene(init='drawer', random=False)
    s2 = s.copy()
    s2.r.eef_position = s.drawer.position

    state = s.generate_scene_state(A, G, U, UID, ('open', 'drawer'))
    bn = MappingBayesNet(state)

    while not done:
        if np.random.uniform(0,1) < epsilon:
            action = (Actions.get_random_action(), 'drawer')
        else:
            bn.create_observation()
            action = bn.select_action()
        print(f"Action: {action}")
        s, reward, done = Actions.step(s, s2, action, ignore_location=False)

        bn.policy_update(reward)


Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: ('open', 'drawer')
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: ('close', 'drawer')
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_left'}
Action cannot be perfo

Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
('move_right', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
('move_right', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
('move_right', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 

Action: {'target_object': 'drawer', 'target_action': 'move_left'}
('move_left', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_left'}
('move_left', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_left'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_up'}
('move_up', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_up'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_up'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_up'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_left'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_left'}
Action cannot be performed!
Action: ('open', 'drawer')
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_down'}
('move_down', 'drawer') done!
Action:

Action: {'target_object': 'drawer', 'target_action': 'move_right'}
('move_right', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
Action cannot be performed!
Action: ('move_up', 'drawer')
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: ('move_right', 'drawer')
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_

Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_left'}
('move_left', 'drawer') done!
Action: ('open', 'drawer')
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
('move_right', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
('move_right', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
('move_right', 'drawer')

...

Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_up'}
('move_up', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_up'}
('move_up', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_up'}
('move_up', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_up'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Act

Action: {'target_object': 'drawer', 'target_action': 'move_left'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_down'}
('move_down', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_down'}
('move_down', 'drawer') done!
Action: ('move_left', 'drawer')
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_up'}
('move_up', 'drawer') done!
Action: {'target_objec

Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: ('move_up', 'drawer')
('move_up', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'open'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target

Action: {'target_object': 'drawer', 'target_action': 'move_left'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_left'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_left'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
('move_right', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'close'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
('move_right', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
('move_right', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 'move_right'}
Action cannot be performed!
Action: {'target_object': 'drawer', 'target_action': 'move_left'}
('move_left', 'drawer') done!
Action: {'target_object': 'drawer', 'target_action': 

KeyboardInterrupt: 